In [ ]:
import pickle
import numpy as np
from tensorflow import keras
import re

In [ ]:
english_dic=open("/content/drive/MyDrive/Fiverr/Romanized Sinhala/english_dic.pickle","rb")
english_dic=pickle.load(english_dic)

inference_dic=open("/content/drive/MyDrive/Fiverr/Romanized Sinhala/inference_dic.pickle","rb")
inference_dic=pickle.load(inference_dic)

In [ ]:
inf_english_dic = {} 
for key in english_dic:
   inf_english_dic[key] = np.argmax(english_dic[key])

In [ ]:
model=keras.models.load_model('/content/drive/MyDrive/Fiverr/Romanized Sinhala/blstm_model.h5')


In [ ]:
def zerolistmaker(n):
    listofzeros = [0] * n
    return listofzeros

In [ ]:
def encoding(string):
  #string="sam2an gedara yai"
  punctuations=zerolistmaker(20)
  words=re.findall(r"[\w']+|[.,!?;]", string)
  words=[list(i) for i in words]
  sentence=np.zeros((len(words),22,60))
  for i,word in enumerate(words):
    if inf_english_dic.get(words[i][0])==None:
      punctuations[i]=words[i][0]
    for j,letter in enumerate(word):
      vectorized=inf_english_dic.get(letter)
      if vectorized == None:
        k = 0
      else:
        k = vectorized
      sentence[i][j][k]=1
    sentence[i][j+1:][:]=1
  return punctuations,sentence

In [ ]:
def translation(string):
  punctuations,encoded=encoding(string)
  results=model.predict(encoded)
  
  numbers=np.argmax(results,axis=2)
  for i, word in enumerate(numbers):
    for j, letter in enumerate(word):
      if word[j]==word[j-1]==59:
        word[j-1:]=0
  l=''
  for i in numbers:
    for j in i:
      l+=inference_dic[j]
  sinhalized=" ".join(l.split())
  sinhalized=sinhalized.split()
  for i,punctuation in enumerate(punctuations):
    if punctuation != 0 and i<=len(sinhalized):
      sinhalized[i] = punctuation

  sinhalized=" ".join(sinhalized)
  sinhalized = " ".join(sinhalized.split())
  return sinhalized
  #return punctuations

In [ ]:
print(translation("aechchita haal garanda"))

අා්චිට හාල් ගරන්ඩ
